In [1]:
import numpy as np
import pandas as pd

# Load data

In [2]:
df = pd.read_csv('ggsn_thp.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   week       616 non-null    int64  
 1   area       616 non-null    object 
 2   ne_thp     616 non-null    float64
 3   volte_vol  616 non-null    float64
 4   rab_user   616 non-null    float64
 5   ggsn       616 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 29.0+ KB


In [3]:
col_week = 'week'
col_area = 'area'
col_nethp = 'ne_thp'
col_volte = 'volte_vol'
col_rabuser = 'rab_user'
col_ggsn = 'ggsn'

col_date = 'date'

In [4]:
df.head()

,week,area,ne_thp,volte_vol,rab_user,ggsn
0,202150,Valparaiso,646.125895,308.484067,2300.921160,229.489611
1,202150,Luang Prabang,648.795419,218.974793,1619.360608,293.011472
2,202150,Chefchaouen,503.692732,412.065085,1733.568785,553.277913
3,202150,Hoi An,479.543186,135.423613,1300.605689,533.289586
4,202150,Antananarivo,423.156395,101.727074,1471.244589,441.462850


In [5]:
df.describe()

,week,ne_thp,volte_vol,rab_user,ggsn
count,616.000000,616.000000,616.000000,616.000000,616.000000
mean,202116.821429,450.702528,234.333467,1567.897937,420.535207
std,27.016055,156.948624,105.187460,511.977004,243.450670
min,202047.000000,64.977835,59.535763,572.206349,63.718254
25%,202107.750000,357.564693,132.889311,1249.448716,232.274627
50%,202121.500000,477.905343,249.200186,1616.140916,407.225813
75%,202136.000000,574.377243,304.286910,1840.411642,516.921587
max,202150.000000,713.875255,441.108184,2689.725152,1655.643375


In [6]:
df.drop(columns=df.describe().columns).describe()

,area
count,616
unique,11
top,Valparaiso
freq,56


# Preprocess

## Basic preprocessing

In [7]:
df[col_week] = df[col_week].astype(str)
df[col_date] = pd.to_datetime(df[col_week] + '0', format='%Y%W%w')
df = df.drop(columns=col_week)
df.head()

,area,ne_thp,volte_vol,rab_user,ggsn,date
0,Valparaiso,646.125895,308.484067,2300.921160,229.489611,2021-12-19
1,Luang Prabang,648.795419,218.974793,1619.360608,293.011472,2021-12-19
2,Chefchaouen,503.692732,412.065085,1733.568785,553.277913,2021-12-19
3,Hoi An,479.543186,135.423613,1300.605689,533.289586,2021-12-19
4,Antananarivo,423.156395,101.727074,1471.244589,441.462850,2021-12-19


## Train test split

In [8]:
datefirst = df[col_date].min()
datelast = df[col_date].max()
print('First date:', datefirst)
print('Last date:', datelast)

datesplit = datefirst + (datelast - datefirst) * 0.8
print('Datesplit:', datesplit)

dftrain = df[df[col_date] <= datesplit]
dftest = df[df[col_date] > datesplit]
print('dftrain shape:', dftrain.shape)
print('dftest shape:', dftest.shape)

First date: 2020-11-29 00:00:00
Last date: 2021-12-19 00:00:00
Datesplit: 2021-10-03 00:00:00
dftrain shape: (506, 6)
dftest shape: (110, 6)


## Partition by area

In [9]:
dictdf = {}
for area, dfgroup in dftrain.groupby(col_area):
    dictdf[area] = dfgroup.copy().sort_values(col_date).reset_index(drop=True)
    
print('Length of dictdf:', len(dictdf))
area = df[col_area].sample(1).iat[0]
print('Sample:',area)
display(dictdf[area].head())
dfdisplay = []
for area in dictdf.keys():
    dfdisplay.append({
        'area': area,
        'nrow': dictdf[area].shape[0],
        'ncol': dictdf[area].shape[1],
        'firstdate': dictdf[area][col_date].min(),
        'lastdate': dictdf[area][col_date].max()
    })
display(pd.DataFrame(dfdisplay))

Length of dictdf: 11
Sample: Valletta


,area,ne_thp,volte_vol,rab_user,ggsn,date
0,Valletta,290.026902,260.160467,1495.504877,237.214800,2020-11-29
1,Valletta,298.352000,267.313700,1536.709042,121.194129,2020-12-06
2,Valletta,304.248937,272.588554,1544.582966,245.532019,2020-12-13
3,Valletta,304.860382,272.206202,1456.717246,258.240449,2020-12-20
4,Valletta,362.780151,324.983423,1822.481473,252.422334,2020-12-27


,area,nrow,ncol,firstdate,lastdate
0,Antananarivo,46,6,2020-11-29,2021-10-03
1,Asmara,46,6,2020-11-29,2021-10-03
2,Chefchaouen,46,6,2020-11-29,2021-10-03
3,Cuenca,46,6,2020-11-29,2021-10-03
4,Essaouira,46,6,2020-11-29,2021-10-03
5,Hoi An,46,6,2020-11-29,2021-10-03
6,Luang Prabang,46,6,2020-11-29,2021-10-03
7,Nafplio,46,6,2020-11-29,2021-10-03
8,Valletta,46,6,2020-11-29,2021-10-03
9,Valparaiso,46,6,2020-11-29,2021-10-03


## Impute missing date, if any

In [13]:
def impute_missing_date(dfarea, date_begin, date_end, column_date, cols, column_area, winsize=7, min_periods=1):
    # construct completed dates
    dates_complete = pd.date_range(date_begin, date_end, freq='W-Sun')
    dfdate = pd.DataFrame({
        col_date: dates_complete
    })

    # merge
    dfi = pd.merge(dfdate, dfarea, how='left', on=column_date)
    for c in cols:
        dfi.loc[dfi[c].isna(), c] = dfi[c]\
            .rolling(window=winsize, min_periods=min_periods)\
            .apply(np.nanmean)
    dfi[column_area] = dfi[column_area].fillna(dfi[column_area].dropna().iat[0])
    
    return dfi

dictdf_imp = {}

for area in dictdf.keys():
    dictdf_imp[area] = impute_missing_date(
        dfarea=dictdf[area],
        date_begin=datefirst,
        date_end=datesplit,
        column_date=col_date,
        cols=[col_ggsn, col_nethp, col_rabuser, col_volte],
        column_area=col_area
    )

print('Data shape after imputing missing dates:')
for area in dictdf_imp.keys():
    print(f'Data shape of {area}: {dictdf_imp[area].shape}, missing: {dictdf_imp[area].isna().sum().sum()}')
area = df[col_area].sample(1).iat[0]
print('Sample:',area)
display(dictdf_imp[area].head())

Data shape after imputing missing dates:
Data shape of Antananarivo: (48, 6), missing: 0
Data shape of Asmara: (48, 6), missing: 0
Data shape of Chefchaouen: (48, 6), missing: 0
Data shape of Cuenca: (48, 6), missing: 0
Data shape of Essaouira: (48, 6), missing: 0
Data shape of Hoi An: (48, 6), missing: 0
Data shape of Luang Prabang: (48, 6), missing: 0
Data shape of Nafplio: (48, 6), missing: 0
Data shape of Valletta: (48, 6), missing: 0
Data shape of Valparaiso: (48, 6), missing: 0
Data shape of Wilmington: (48, 6), missing: 0
Sample: Asmara


,date,area,ne_thp,volte_vol,rab_user,ggsn
0,2020-11-29,Asmara,525.955282,252.505903,1746.331908,920.684170
1,2020-12-06,Asmara,534.183494,256.319922,1754.474737,958.306765
2,2020-12-13,Asmara,536.714487,256.595475,1728.561094,958.151806
3,2020-12-20,Asmara,536.744002,255.674059,1625.285332,964.544942
4,2020-12-27,Asmara,532.054635,254.101456,1679.777163,984.101318


## Outlier detection

In [16]:
def detect_outlier(xs, winsize=5, dispersion_factor = 2, verbose=False, pos=None):
    
    if pos is None:
        pos = 'center'
    
    if pos == 'center':
        center_idx = winsize // 2
        
        if (len(xs) < winsize):
            center_idx = len(xs) - int(np.ceil(winsize / 2))
    elif pos == 'last':
        center_idx = winsize - 1
    else:
        raise Exception('pos argument only accepts None, \'center\', or \'last\'')
    
    
    center = xs[center_idx]
    
    # Outlier method 1: window median
    med = np.nanmedian(xs)
    dist_to_med = center - med
    
    #avg = np.nanmean(xs)
    #dist_to_mean = center - avg

    iqr = np.percentile(xs, 75)- np.percentile(xs, 25)
    
    local_med_outlier = (np.abs(dist_to_med) > dispersion_factor * iqr)*1
    #local_mean_outlier = (np.abs(dist_to_mean) > dispersion_factor * iqr)*1
    
    bef = xs[:center_idx]
    aft = xs[center_idx:]
    
    # Outlier method 2: Median of points before center
    med_bef = np.nanmedian(bef)
    iqr_bef = np.percentile(bef, 75)- np.percentile(bef, 25)
    
    # Outlier method 3: Median of points after center
    med_aft = np.nanmedian(aft)
    iqr_aft = np.percentile(aft, 75)- np.percentile(aft, 25)
    
    bef_med_outlier = (np.abs(center - med_bef) > dispersion_factor * iqr_bef)*1
    aft_med_outlier = (np.abs(center - med_aft) > dispersion_factor * iqr_aft)*1
    
    outlier_score = np.mean([local_med_outlier, bef_med_outlier, aft_med_outlier])
                    
    if verbose:
        print(f'xs: {xs}')
        print(f'len(xs): {len(xs)}')
        print(f'center_idx: {center_idx}')
        print(f'med: {med}')
        print(f'dist_to_med: {dist_to_med}')
        print(f'iqr: {iqr}')
        print(f'med_bef: {med_bef}')
        print(f'iqr_bef: {iqr_bef}')
        print(f'med_aft: {med_aft}')
        print(f'iqr_aft: {iqr_aft}')
        print(f'local_med_outlier: {local_med_outlier}')
        print(f'bef_med_outlier: {bef_med_outlier}')
        print(f'aft_med_outlier: {aft_med_outlier}')
        print(f'outlier_score: {outlier_score}')
        print('')
        
    return(outlier_score)

winsize = 7

dictdf_out = {}

for area in dictdf_imp.keys():
    dfo = dictdf_imp[area].copy()
    
    # on col_ggsn
    dfo[f'{col_ggsn}_outlier'] =  dfo[col_ggsn].rolling(window=winsize, center=True)\
            .apply(detect_outlier, raw=True, kwargs={'winsize': winsize,
                                                     'dispersion_factor': 1.5,
                                                     'verbose': False})
    
    dfo[f'{col_ggsn}_outlier'] = dfo[f'{col_ggsn}_outlier'].fillna(
        dfo[col_ggsn].rolling(window=winsize, center=False)\
            .apply(detect_outlier, raw=True, kwargs={'winsize': winsize,
                                                     'dispersion_factor': 1.5,
                                                     'verbose': False}))
    
    dfo[f'{col_ggsn}_outlier'] = dfo[f'{col_ggsn}_outlier'].fillna(
        dfo[col_ggsn].iloc[::-1].rolling(window=winsize, center=False)\
            .apply(detect_outlier, raw=True, kwargs={'winsize': winsize,
                                                     'dispersion_factor': 1.5,
                                                     'verbose': False})\
                .iloc[::-1])
    
    # Set default override flag to False
    dfo[f'{col_ggsn}_outlier_override'] = False
    
    # on col_nethp
    dfo[f'{col_nethp}_outlier'] =  dfo[col_nethp].rolling(window=winsize, center=True)\
            .apply(detect_outlier, raw=True, kwargs={'winsize': winsize,
                                                     'dispersion_factor': 1.5,
                                                     'verbose': False})
    
    dfo[f'{col_nethp}_outlier'] = dfo[f'{col_nethp}_outlier'].fillna(
        dfo[col_nethp].rolling(window=winsize, center=False)\
            .apply(detect_outlier, raw=True, kwargs={'winsize': winsize,
                                                     'dispersion_factor': 1.5,
                                                     'verbose': False}))
    
    dfo[f'{col_nethp}_outlier'] = dfo[f'{col_nethp}_outlier'].fillna(
        dfo[col_nethp].iloc[::-1].rolling(window=winsize, center=False)\
            .apply(detect_outlier, raw=True, kwargs={'winsize': winsize,
                                                     'dispersion_factor': 1.5,
                                                     'verbose': False})\
                .iloc[::-1])
    
    # Set default override flag to False
    dfo[f'{col_nethp}_outlier_override'] = False
    
    dictdf_out[area] = dfo.copy()
    
del dfo

for area in dictdf_out.keys():
    print(f'{area} shape before: {dictdf_imp[area].shape}, after: {dictdf_out[area].shape}')

Antananarivo shape before: (48, 6), after: (48, 10)
Asmara shape before: (48, 6), after: (48, 10)
Chefchaouen shape before: (48, 6), after: (48, 10)
Cuenca shape before: (48, 6), after: (48, 10)
Essaouira shape before: (48, 6), after: (48, 10)
Hoi An shape before: (48, 6), after: (48, 10)
Luang Prabang shape before: (48, 6), after: (48, 10)
Nafplio shape before: (48, 6), after: (48, 10)
Valletta shape before: (48, 6), after: (48, 10)
Valparaiso shape before: (48, 6), after: (48, 10)
Wilmington shape before: (48, 6), after: (48, 10)
